In [ ]:
!pip install kaggle  # Install Kaggle CLI
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d ashwingupta3012/male-and-female-faces-dataset

!unzip male-and-female-faces-dataset.zip -d /content/dataset/


Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/Male and Female face dataset/Female Faces/0 (1367).jpg  
  inflating: /content/dataset/Male and Female face dataset/Female Faces/0 (1368).jpg  
  inflating: /content/dataset/Male and Female face dataset/Female Faces/0 (1369).jpg  
  inflating: /content/dataset/Male and Female face dataset/Female Faces/0 (137).jpg  
  inflating: /content/dataset/Male and Female face dataset/Female Faces/0 (1370).jpg  
  inflating: /content/dataset/Male and Female face dataset/Female Faces/0 (1371).jpg  
  inflating: /content/dataset/Male and Female face dataset/Female Faces/0 (1372).jpg  
  inflating: /content/dataset/Male and Female face dataset/Female Faces/0 (1373).jpg  
  inflating: /content/dataset/Male and Female face dataset/Female Faces/0 (1374).jpg  
  inflating: /content/dataset/Male and Female face dataset/Female Faces/0 (1375).jpg  
  inflating: /content/dataset/Male and Female face dataset/Female Faces/0 (1376)

In [ ]:
!pip install -q --upgrade torch_snippets
from torch_snippets import *
import torchvision, torch
from torchvision import transforms
import torchvision.utils as vutils
import cv2, numpy as np, pandas as pd
import torch.nn as nn
device = "cuda" if torch.cuda.is_available() else "cpu"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.3 MB/s eta 0:00:00


In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + \
 'haarcascade_frontalface_default.xml')

In [ ]:
!mkdir cropped_faces
images = Glob('/content/dataset/Male and Female face dataset/Female Faces/0*.jpg') + Glob('/content/dataset/Male and Female face dataset/Male Faces/1*.jpg')
for i in range(len(images)):
 img = read(images[i],1)
 gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 faces = face_cascade.detectMultiScale(gray, 1.3, 5)
 for (x,y,w,h) in faces:
  img2 = img[y:(y+h),x:(x+w),:]
 cv2.imwrite('cropped_faces/'+str(i)+'.jpg', \
 cv2.cvtColor(img2, cv2.COLOR_RGB2BGR))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
save_dir = '/content/drive/MyDrive/cropped_faces'
os.makedirs(save_dir, exist_ok=True)

cv2.imwrite(f'{save_dir}/{i}.jpg', cv2.cvtColor(img2, cv2.COLOR_RGB2BGR))

# (drive.mount('/content/drive')), and your cropped faces will be available in '/content/drive/MyDrive/cropped_faces/'.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


True

In [ ]:
#@ transforms:
transform=transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
#@ DataLoader:
from torch.utils.data import Dataset, DataLoader
class Faces(Dataset):
  def __init__(self, folder):
    super().__init__()
    self.folder=folder
    self.images=sorted(Glob(folder))

  def __len__(self):
    return len(self.images)

  def __getitem__(self, ix):
    image_path=self.images[ix]
    image=Image.open(image_path)
    image=transform(image)
    return image


In [ ]:
dataset=Faces(folder='cropped_faces/')

In [ ]:
dataloader=DataLoader(dataset, batch_size=64, shuffle=True, num_workers=8)

In [2]:
def weights_init(m):
  classname=m.__class__.__name__ #checks what type of layer are we dealing
  if classname.find('Conv')!=-1:
    nn.init.normal_(m.weight.data, 0.0, 0.02) #0.0=mean, 0.02=std
  elif classname.find('BatchNorm')!= -1:
    nn.init.normal_(m.weight.data, 1.0, 0.02)
    nn.init.constant_(m.bias.data, 0)

In [ ]:
#@ discriminator:
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()
    self.model=nn.Sequential(
        nn.Conv2d(3, 64, 4, 2, 1, bias=False),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Conv2d(64, 64*2, 4, 2, 1, bias=False),
        nn.BatchNom2d(64*2),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Conv2d(64*2, 64*4, 4, 2, 1, bias=False),
        nn.BatchNom2d(64*4),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Conv2d(64*4, 64*8, 4, 2, 1, bias=False),
        nn.BatchNom2d(64*8),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Conv2d(64*8, 1, 4, 1, bias=False),
        nn.Sigmoid()
    )